<a href="https://colab.research.google.com/github/Nowitkiewicz/PortfolioProjects/blob/main/dog_vision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow
from IPython.display import Image
labels = pd.read_csv("/content/drive/MyDrive/dog_vision/labels.csv")

#filenames = os.listdir("/content/drive/MyDrive/dog_vision/train/")
#filenames = ['drive/MyDrive/dog-vision/train/' + fname + '.jpg' for fname in labels['id']]
#fileBreed =["drive/MyDrive/dog-vision/train/" + fname for fname in labels['breed']]

#from matplotlib.pyplot import imread, imshow
#image = imread(filenames[1])
#imshow(image)

In [ ]:
uniquebreeds = np.unique(fileBreed)

In [ ]:
uniquebreeds == fileBreed[0])

In [ ]:
import numpy as np
uniquebreeds = np.unique(fileBreed )
#print(uniquebreeds = fileBreed)
print(fileBreed[0]== uniquebreeds)
#print(uniquebreeds[19])
#print(len(uniquebreeds))
#rint(fileBreed[0])

[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False]


In [ ]:
boolean_labels = [label == uniquebreeds for label in fileBreed]
len(boolean_labels)

In [ ]:
print(fileBreed[0])
print(np.where(uniquebreeds == fileBreed[0]))
# print(boolean_labels[0].argmax())
#print(boolean_labels[0].astype(tf.float32))
#print(boolean_labels[0])

/content/drive/MyDrive/dog_vision/train/boston_bull
(array([19]),)


In [ ]:
print(fileBreed[0])
print(np.where(uniquebreeds == fileBreed[0]))
print(boolean_labels[0].argmax())
print(boolean_labels[0].astype(int))

/content/drive/MyDrive/dog_vision/train/boston_bull
(array([19]),)
19
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [ ]:
print(fileBreed[0])
print(np.where(uniquebreeds == fileBreed[0]))
print(boolean_labels[0].argmax())
print(boolean_labels[0].astype(int))

/content/drive/MyDrive/dog_vision/train/boston_bull
(array([19]),)
19
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [ ]:
boolean_labels = [label == uniquebreeds for label in fileBreed]

boolean_labels[0],uniquebreeds[0]

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]),
 '/content/drive/MyDrive/dog_vision/train/affe

In [ ]:
print(fileBreed[0])
print(np.where(uniquebreeds == fileBreed[0]))
print(boolean_labels[0].argmax())
print(boolean_labels[0].astype(int))

/content/drive/MyDrive/dog_vision/train/boston_bull
(array([19]),)
19
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [ ]:
from sklearn.model_selection import train_test_split
X = filenames
y = boolean_labels
X_train,X_val,y_train,y_val = train_test_split(X, y , test_size = 0.2)



Preprocessing image to tensor 1.take an image file as input 2.use tensorflow to read and save to variable image 3.turn image (jpg) into Tensors 4.resize image to 224,224



In [ ]:
tensor = tf.constant(image)
tensor

In [ ]:
from matplotlib.pyplot import imread
image = imread(filenames[42])
imshow(image)

In [ ]:
#define image size
IMG_SIZE = 224

#Creating function for preprocessing image

def process_image(image_path,img_size = IMG_SIZE):
    """ Takes an image from image_path and turns it into tensors
    """
    #Read an image file
    image = tf.io.read_file(image_path)

    #Take an image ( jpg) and turn it into numbers
    image = tf.image.decode_jpeg(image, channels = 3)

    #Convert color channel from 0-255 to 0-1
    image = tf.image.convert_image_dtype(image, tf.float32)

    #Resize the image to our desired value
    image = tf.image.resize(image, size = [IMG_SIZE,IMG_SIZE])

    return image

In [ ]:
image = tf.io.read_file(filenames[0])

In [ ]:
image = tf.image.decode_jpeg(image, channels = 3)
image

In [ ]:
IMG_SIZE = 224
image = tf.image.convert_image_dtype(image, tf.float32)

In [ ]:
image

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.5705322 , 0.33588538, 0.15008473],
        [0.66901225, 0.4601166 , 0.26022965],
        [0.61062384, 0.43763596, 0.21601713],
        ...,
        [0.8182312 , 0.7297958 , 0.26963907],
        [0.72454786, 0.6059109 , 0.2653059 ],
        [0.7494603 , 0.6252918 , 0.32456532]],

       [[0.6761415 , 0.4733761 , 0.30642927],
        [0.5672121 , 0.38192675, 0.20349538],
        [0.5883547 , 0.4322571 , 0.2403155 ],
        ...,
        [0.9326942 , 0.8260503 , 0.41093278],
        [0.86072475, 0.74250627, 0.3694122 ],
        [0.76781666, 0.6461604 , 0.29291284]],

       [[0.46350825, 0.29595745, 0.1536156 ],
        [0.42554107, 0.27039403, 0.12009468],
        [0.7887875 , 0.6547362 , 0.49015   ],
        ...,
        [0.89372385, 0.7696401 , 0.41811958],
        [0.88609535, 0.7585918 , 0.36495727],
        [0.8243601 , 0.69494826, 0.2930575 ]],

       ...,

       [[0.7344025 , 0.4716574 , 0.2559711 ],
        [0.71

In [ ]:
image = tf.image.resize(image, size = [IMG_SIZE,IMG_SIZE])

In [ ]:
boolean_labels[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:
#process_image(X[0])
#image
#z = [label == boolean_labels[0] for label in fileBreed if label == boolean_labels[0]]
for label in boolean_labels:
    #if label == boolean_labels[0]:
    print(label)

NameError: ignored

Turning our data into batches.


Create a function to return a tuple (image,label)

In [ ]:
def get_image_label(X, y):
    """Takes an image filepath name and assoassociated label,
    processes the image and return a tuple (image,label)
    """
    image = process_image(X)

    return image, y

In [ ]:
get_image_label(X[0],y[0])
#print(boolean_labels[0].argmax())
z = label == boolean_labels[0]
#y[0] == np.argmax()
z

NameError: ignored

Let's turn all of our data into batches.

In [ ]:
#Define batch size of 32
BATCH_SIZE = 32
#Create a function to create a batches from given data.
def create_data_batches(X, y = None , batch_size = BATCH_SIZE,valid_data = False, test_data = False):
    """ Create batches of data out of image (X)  and label (y) pairs.
    Shuffles the data if it's training data but doesn't shuffle if it's validation data.
    Also accepts test data as input (no label)
    """
    #If the data is test dataset we probably don't have labels
    if test_data:
        print("Creating test databatches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) #only filepaths
        data_batch = data.map(process_image).batch(BATCH_SIZE)
        return data_batch

    #if the data is a valid set we don't need to shuffle it.
    elif valid_data:
        print("Creating validation databatches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), #filepaths
                                                   tf.constant(y))) #labels

        data_batch = data.map(get_image_label).batch(BATCH_SIZE)

        return data_batch

    #Turn filepaths and labels into tensors.
    else:
        print("Creating training databatches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                 tf.constant(y)))
        #Shuffling data and labels.
        data = data.shuffle(buffer_size = len(X))
        data = data.map(get_image_label)
        data_batch = data.batch(BATCH_SIZE)

    return data_batch


In [ ]:
create_data_batches(X, y , batch_size = BATCH_SIZE,valid_data = True, test_data = False)

Creating validation databatches...


<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>

In [ ]:
create_data_batches(X, y  , batch_size = BATCH_SIZE,valid_data = False, test_data = False)

Creating training databatches...


<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 120), dtype=tf.bool, name=None))>